In [2]:
from kafka import KafkaProducer
import json
import requests
import time

# Define Kafka Producer
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
)

KRAKEN_API_URL = "https://api.kraken.com/0/public/Ticker"

# Crypto pairs you are tracking
crypto_pairs = ["ADAUSD", "DOTUSD", "LINKUSD", "MATICUSD", "SOLUSD",
                "XDGUSD", "XETHZUSD", "XLTCZUSD", "XXBTZUSD", "XXRPZUSD"]

def fetch_trade_data():
    """Fetch real-time trade data from Kraken API and send to Kafka."""
    response = requests.get(KRAKEN_API_URL, params={"pair": ",".join(crypto_pairs)})
    data = response.json().get("result", {})

    for pair, details in data.items():
        trade_data = {
            "pair": pair,
            "price": float(details["c"][0]),  # Last trade closed price
            "volume": float(details["v"][0]),  # 24-hour volume
            "timestamp": int(time.time()),
        }
        producer.send("crypto_trades", trade_data)
        print(f"Sent: {trade_data}")

while True:
    fetch_trade_data()
    time.sleep(10)  # Fetch every 10 seconds


Sent: {'pair': 'ADAUSD', 'price': 0.741447, 'volume': 23265200.92492771, 'timestamp': 1738703018}
Sent: {'pair': 'DOTUSD', 'price': 4.66, 'volume': 1035045.17156957, 'timestamp': 1738703018}
Sent: {'pair': 'XETHZUSD', 'price': 2709.78, 'volume': 36697.97433637, 'timestamp': 1738703018}
Sent: {'pair': 'LINKUSD', 'price': 19.80519, 'volume': 455889.12944513, 'timestamp': 1738703018}
Sent: {'pair': 'XLTCZUSD', 'price': 100.23, 'volume': 67608.29868087, 'timestamp': 1738703018}
Sent: {'pair': 'MATICUSD', 'price': 0.3064, 'volume': 916457.99032747, 'timestamp': 1738703018}
Sent: {'pair': 'SOLUSD', 'price': 205.31, 'volume': 324563.93490592, 'timestamp': 1738703018}
Sent: {'pair': 'XXBTZUSD', 'price': 98137.7, 'volume': 2511.54216402, 'timestamp': 1738703018}
Sent: {'pair': 'XDGUSD', 'price': 0.2604595, 'volume': 161758847.14448255, 'timestamp': 1738703018}
Sent: {'pair': 'XXRPZUSD', 'price': 2.51931, 'volume': 60634926.50653313, 'timestamp': 1738703018}
Sent: {'pair': 'ADAUSD', 'price': 0.7

KeyboardInterrupt: 